### pip install

In [51]:
#pip install
!pip install faiss-gpu --quiet
!pip install boto3 --quiet
!pip install langchain-aws --quiet
!pip install langchain-community boto3 --quiet
!pip install boto3 requests requests-aws4auth --quiet
!pip install pymongo --quiet
!pip install google-search-results sagemaker --quiet
!pip install boto3 nltk --quiet
!pip install wikipedia --quiet
!pip install tavily-python --quiet
!pip install --upgrade langchain sympy numexpr --quiet
!pip install langchain-experimental --quiet
!pip install -U langgraph langchain_community langchain_anthropic langchain_experimental --quiet
!pip install langchain-redis langchain-openai redis --quiet
#!docker run -d -p 6379:6379 redis:latest --quiet

### libraries

In [52]:
import boto3
import os
from langchain import hub

#chat models
from langchain_aws import ChatBedrock
from langchain_aws import ChatBedrockConverse

#LLMs
from langchain_aws import BedrockLLM
from langchain_community.llms import AmazonAPIGateway
from langchain_aws import SagemakerEndpoint

#prompts
from langchain.prompts import PromptTemplate

#embedding models
from langchain_aws import BedrockEmbeddings
# from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import ContentHandlerBase

#document loaders
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
from langchain_community.document_loaders import AmazonTextractPDFLoader

#vector stores
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_community.vectorstores import DocumentDBVectorSearch

from langchain_aws.vectorstores.inmemorydb import InMemoryVectorStore

#retrievers
from langchain_aws import AmazonKendraRetriever
from langchain_aws import AmazonKnowledgeBasesRetriever

#memory
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables import RunnablePassthrough

#graphs
from langchain_community.graphs import NeptuneGraph
from langchain_community.graphs import NeptuneAnalyticsGraph
from langchain_community.chains.graph_qa.neptune_cypher import NeptuneOpenCypherQAChain

from langchain_community.graphs import NeptuneRdfGraph
from langchain_community.chains.graph_qa.neptune_sparql import NeptuneSparqlQAChain

#callbacks
from langchain_community.callbacks.bedrock_anthropic_callback import BedrockAnthropicTokenUsageCallbackHandler
from langchain_community.callbacks import SageMakerCallbackHandler

#chains
# from langchain_experimental.comprehend_moderation import AmazonComprehendModerationChain

#tavily
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults

#tools
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.tools import BaseTool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, CSVLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

#agent
from langchain.agents import create_tool_calling_agent, AgentType
from langchain.agents import AgentExecutor
from langchain.chat_models import ChatOpenAI

In [53]:
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 200}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3.client('bedrock'),
    model_kwargs=model_parameter, 
    beta_use_converse_api=True
)


In [54]:
os.environ["TAVILY_API_KEY"] = "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"
search = TavilySearchResults()

In [55]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    """You are an assistant for question-answering tasks. 
    be concise
    \n\n
    {agent_scratchpad}
    "context": history
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input},{agent_scratchpad}"),
    ]
)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [56]:
# Import necessary libraries
# !pip install -U wikipedia
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.tools import BaseTool

loaded_tools = load_tools(["llm-math", "wikipedia"], llm=bedrock_llm)
tools = [search,loaded_tools[0],loaded_tools[1]]


In [57]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langgraph.checkpoint.memory import MemorySaver

agent = create_tool_calling_agent(bedrock_llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [58]:
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables import RunnablePassthrough

In [59]:
!pip install langchain-redis langchain-openai redis
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory
!docker run -d -p 6379:6379 redis:latest

7bf6b93573f8222eddc6bec0ed2ee422ec7fb0675c5c50e7d7a80ea8267eff18
docker: Error response from daemon: driver failed programming external connectivity on endpoint cool_villani (fffcff78a5693300002dab80ceff895886ec19314d43937e35c4b4f2f89f3b84): Bind for 0.0.0.0:6379 failed: port is already allocated.


In [60]:
# Replace with your Redis server URL and session ID
redis_url = "redis://localhost:6379/0"
session_id = "foo"

# Initialize RedisChatMessageHistory
history = RedisChatMessageHistory(session_id=session_id, url=redis_url)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

# Create a prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

# Initialize the language model
llm = bedrock_llm

# Create the conversational chain
chain = prompt | llm

# Function to get or create a RedisChatMessageHistory instance
def get_redis_history(session_id: str) -> BaseChatMessageHistory:
    return RedisChatMessageHistory(session_id, redis_url)

# Create a runnable with message history
agent_with_chat_history = RunnableWithMessageHistory(
    chain,
    get_redis_history, 
    input_messages_key="input", 
    history_messages_key="history"
)

response1 = agent_with_chat_history.invoke(
    {"input": "What's the population of Canada?"},
    config={"configurable": {"session_id": session_id}},
)

print("AI Response1:", response1.content)


response2 = agent_with_chat_history.invoke(
    {"input": "What is their anthem?"},
    config={"configurable": {"session_id": session_id}},
)

print("AI Response2:", response2.content)

AI Response1: The current population of Canada is around 38.25 million as of 2022 estimates.

Some additional details about Canada's population:

- Canada is the 38th most populous country in the world.

- The most populous province is Ontario with about 14.8 million people, followed by Quebec with around 8.6 million.

- Around 86% of Canadians live in urban areas, making Canada one of the most urbanized countries in the world.

- The largest metropolitan area is Toronto with over 6.2 million residents.

- Canada has an aging population, with a median age of around 41 years old.

- Immigration is a major driver of Canada's population growth, with over 300,000 new permanent residents admitted annually in recent years.

So in summary, while not an extremely populous nation globally, Canada has a sizeable population concentrated mainly in urban centers, especially in the provinces of Ontario and Quebec.
AI Response2: The national anthem of Canada is "O Canada".

Here are some key details 